In [572]:
from typing import List, Dict

import numpy as np
from mdptoolbox import mdp
from itertools import product
from scipy.sparse import csr_matrix

In [573]:
# data
field_content = ['empty', 'white', 'blue', 'red']
task = ['store', 'restore']
item = ['white', 'blue', 'red']
task_and_item = [['store', 'white'], ['store', 'blue'], ['store', 'red'],
                 ['restore', 'white'], ['restore', 'blue'], ['restore', 'red']]
actions = [0, 1, 2, 3]

"""
2x2 warehouse with reward/distance at every position:

                [3/4 1/6]
in/out_item ->  [5/2 3/4]
"""
rewards_dict = {0: 4, 1: 3, 2: 2, 3: 1}
dist_dict = {0: 2, 1: 4, 2: 4, 3: 6}

In [574]:
def probabilities_from_data() -> Dict:
    """
    Returns a dictionary containing the relative frequencies of every task/item combination of trainingdata.txt.
    """

    # read in data
    f = open('trainingdata.txt', 'r')

    # convert every line to [<task>, <item>] array and store in color_and_tasks.
    color_and_tasks = []
    line = f.readline()
    while line:
        color_and_tasks.append(line.split())
        line = f.readline()

    total_length = len(color_and_tasks)

    # compute relative frequencies.
    p_store_white = len([cur for cur in color_and_tasks if cur == ['store', 'white']]) / total_length
    p_store_blue = len([cur for cur in color_and_tasks if cur == ['store', 'blue']]) / total_length
    p_store_red = len([cur for cur in color_and_tasks if cur == ['store', 'red']]) / total_length
    p_restore_white = len([cur for cur in color_and_tasks if cur == ['restore', 'white']]) / total_length
    p_restore_blue = len([cur for cur in color_and_tasks if cur == ['restore', 'blue']]) / total_length
    p_restore_red = len([cur for cur in color_and_tasks if cur == ['restore', 'red']]) / total_length

    # check if relative frequencies sum up to 1.
    np.testing.assert_equal(
        p_store_white + p_store_blue + p_store_red + p_restore_white + p_restore_blue + p_restore_red,
        1)

    # store in dictionary.
    return {'store_white': p_store_white,
            'store_blue': p_store_blue,
            'store_red': p_store_red,
            'restore_white': p_restore_white,
            'restore_blue': p_restore_blue,
            'restore_red': p_restore_red}


probs = probabilities_from_data()
print(probs)

{'store_white': 0.1278493557978196, 'store_blue': 0.12528906508093823, 'store_red': 0.24686157912124215, 'restore_white': 0.1278493557978196, 'restore_blue': 0.12528906508093823, 'restore_red': 0.24686157912124215}


In [575]:
def get_states() -> List[str]:
    """
    Computes and returns the states. A state has the form

    [<field_content>[0], <field_content>[1], <field_content>[2], <field_content>[3], <task>, <item>]

    and represents an <item> to be stored/restored (represented by <task>) in the warehouse

    [<field_content>[2] <field_content>[3]]
    [<field_content>[0] <field_content>[1]].
    """
    return list(product(field_content, field_content, field_content, field_content, task, item))

In [576]:
states = get_states()
print(states)

num_states = len(states)

# check if all elements are unique.
np.testing.assert_equal(len(list(set(states))), num_states)

[('empty', 'empty', 'empty', 'empty', 'store', 'white'), ('empty', 'empty', 'empty', 'empty', 'store', 'blue'), ('empty', 'empty', 'empty', 'empty', 'store', 'red'), ('empty', 'empty', 'empty', 'empty', 'restore', 'white'), ('empty', 'empty', 'empty', 'empty', 'restore', 'blue'), ('empty', 'empty', 'empty', 'empty', 'restore', 'red'), ('empty', 'empty', 'empty', 'white', 'store', 'white'), ('empty', 'empty', 'empty', 'white', 'store', 'blue'), ('empty', 'empty', 'empty', 'white', 'store', 'red'), ('empty', 'empty', 'empty', 'white', 'restore', 'white'), ('empty', 'empty', 'empty', 'white', 'restore', 'blue'), ('empty', 'empty', 'empty', 'white', 'restore', 'red'), ('empty', 'empty', 'empty', 'blue', 'store', 'white'), ('empty', 'empty', 'empty', 'blue', 'store', 'blue'), ('empty', 'empty', 'empty', 'blue', 'store', 'red'), ('empty', 'empty', 'empty', 'blue', 'restore', 'white'), ('empty', 'empty', 'empty', 'blue', 'restore', 'blue'), ('empty', 'empty', 'empty', 'blue', 'restore', 'red'

In [577]:
def field_content_equals(from_state: list, to_state: list) -> bool:
    """
    Indicates whether the field_content part of the from_state and the to_state
    are the same.
    """
    return from_state[:4] == to_state[:4]

In [578]:
def transition_prob(action: int, from_state: list, to_state: list) -> float:
    """
    Computes the transition probability between a from_state and a to_state executing
    an action.

    :param action: the action to be executed (lies in [0, 1, 2, 3])
    :param from_state: from-state of the transition
    :param to_state: to-state of the transition
    :return: Transition probability
    """

    # get a list version of the from_state to compare it later with the to_state.
    copy_from_state = list(from_state)

    # grab the item part of the to_state.
    next_item = to_state[-1]

    if copy_from_state[4] == 'store':
        # if this transition is not valid (warehouse location occupied)
        if copy_from_state[action] != 'empty':
            # agent stays in the from_state -> probability = 1
            return 1 if from_state == to_state else 0

        # if the applied action results in the to_state, return probability for that
        copy_from_state[action] = copy_from_state[-1]
        return probs['store_' + next_item] if field_content_equals(copy_from_state, list(to_state)) else 0
    elif copy_from_state[4] == 'restore':
        # if this transition is not valid (warehouse location is empty)
        if copy_from_state[action] != copy_from_state[-1]:
            # agent stays in the from_state -> probability = 1
            return 1 if from_state == to_state else 0

        # if the applied action results in the to_state, return probability for that
        copy_from_state[action] = 'empty'
        return probs['restore_' + next_item] if field_content_equals(copy_from_state, list(to_state)) else 0

    return 0

In [579]:
# test transition probabilities

test_prob_impossible_restore_stay = transition_prob(0, ['empty', 'empty', 'empty', 'empty', 'restore', 'red'],
                                                    ['empty', 'empty', 'empty', 'empty', 'restore', 'red'])

np.testing.assert_equal(test_prob_impossible_restore_stay, 1)

test_prob_impossible_store_no_stay = transition_prob(0, ['blue', 'empty', 'empty', 'empty', 'restore', 'red'],
                                                     ['blue', 'red', 'empty', 'empty', 'restore', 'red'])

np.testing.assert_equal(test_prob_impossible_store_no_stay, 0)

test_prob_possible_store = transition_prob(1, ['blue', 'empty', 'empty', 'empty', 'store', 'red'],
                                           ['blue', 'red', 'empty', 'empty', 'store', 'blue'])

np.testing.assert_equal(test_prob_possible_store, probs['store_blue'])

In [580]:
def reward(action: int, state: List[str]) -> float:
    if state[action] != 'empty':
        return rewards_dict[action]

    return 0

In [581]:
def transition_and_reward_matrix():
    """
    Computes the transition matrix as a scipy.sparse.csr_matrix and the reward matrix.

    :return: transition and reward matrix
    """
    transitions = []
    rewards = []

    # compute the transition matrix and reward vector for every action.
    for action in actions:
        row = []
        col = []
        data = []

        reward_vector = []

        # compute the transition probability between every state and append it to transitions.
        for id_from, from_state in enumerate(states):
            for id_to, to_state in enumerate(states):
                p = transition_prob(action, from_state, to_state)

                if p > 0:
                    row.append(id_from)
                    col.append(id_to)
                    data.append(p)

            # append the reward of every state to rewards.
            reward_vector.append(reward(action, from_state))

        transitions.append(csr_matrix((data, (row, col)), shape=(num_states, num_states)))
        rewards.append(reward_vector)

    # transpose rewards since the mdptoolbox expects a (S, A) reward matrix
    return transitions, np.array(rewards).T

In [582]:
P, R = transition_and_reward_matrix()

In [583]:
P[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [584]:
R

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [4, 3, 2, 1],
       [4, 3, 2, 1],
       [4, 3, 2, 1]])

In [585]:
# test if transition matrices are stochastic matrices.
ones = np.zeros((4, num_states))
for i, m in enumerate(P):
    ones[i] = np.sum(np.array(m.toarray()), axis=1)

np.testing.assert_array_almost_equal(ones, np.ones_like(ones))

In [586]:
# test if shape of reward matrix is correct.
np.testing.assert_array_equal(R.shape, (num_states, 4))

In [587]:
algorithms = [mdp.PolicyIteration, mdp.PolicyIterationModified]
discounts = [0.3, 0.5, 0.7, 0.9]

policies = []
for alg in algorithms:
    for d in discounts:
        pass
        # pi = alg(P, R, d)
        # pi.run()

        # policies.append(pi.policy)

In [588]:
pi = mdp.PolicyIteration(P, R, 0.3)
pi.run()

In [589]:
result_policy = pi.policy
print(len(result_policy))

1536


In [590]:
def next_comb_mdp(cur_state: List, state_index: int, policy: tuple) -> List:
    """
    Computes the next field_content part starting in the cur_state and following the policy.

    :param cur_state: the previous state.
    :param state_index: index in states of cur_state. Used to find the value in policy.
    :param policy: the policy to be followed.
    :return: field_content part as List.
    """

    action = policy[state_index]
    result = cur_state[:4]
    cur_task = cur_state[4]
    cur_item = cur_state[-1]

    if cur_task == 'store':
        result[action] = cur_item
    elif cur_task == 'restore':
        result[action] = 'empty'

    return result

In [591]:
def distance_mdp(warehouse_input: List[tuple], policy: tuple) -> float:
    """
    Computes the distance the robot goes, given the warehouse_input and following the policy.

    :param warehouse_input: List of task/item combinations that are coming to the warehouse.
    :param policy: the policy to be followed.
    :return: total distance the robot goes.
    """

    # start with empty warehouse
    cur_comb = ['empty', 'empty', 'empty', 'empty']
    total_dist = 0

    # work off the warehouse_input
    for cur_task, cur_item in warehouse_input:
        cur_comb.append(cur_task)
        cur_comb.append(cur_item)

        # find transition index for policy
        state_index = states.index(tuple(cur_comb))

        # compute next field_content part and add distance to total_dist
        cur_comb = next_comb_mdp(cur_comb, state_index, policy)
        total_dist += dist_dict[policy[state_index]]

    return total_dist

In [592]:
def next_comb_greedy(cur_state: List) -> List:
    """
    Computes the next field_content part starting in the cur_state following a greedy approach.
    
    :param cur_state: the previous state.
    :return: field_content part as List.
    """
    
    result = cur_state[:4]
    cur_task = cur_state[4]
    cur_item = cur_state[-1]

    if cur_task == 'store':
        # find next empty field
        for action, field in enumerate(result):
            if field == 'empty':
                result[action] = cur_item
                return result, action

    elif cur_task == 'restore':
        # find next field with cur_item
        for action, field in enumerate(result):
            if field == cur_item:
                result[action] = 'empty'
                return result, action

    return result, 0

In [593]:
def distance_greedy(warehouse_input: List[tuple]) -> float:
    """
    Computes the distance the robot goes, given the warehouse_input and following a greedy approach.

    :param warehouse_input: List of task/item combinations that are coming to the warehouse.
    :return: total distance the robot goes.
    """
    
    # start with empty warehouse
    cur_comb = ['empty', 'empty', 'empty', 'empty']
    total_reward = 0

    # work off the warehouse_input
    for cur_task, cur_item in warehouse_input:
        cur_comb.append(cur_task)
        cur_comb.append(cur_item)

        # compute next field_content part and add distance to total_dist
        cur_comb, action = next_comb_greedy(cur_comb)
        total_reward += dist_dict[action]

    return total_reward

In [594]:
def data_to_warehouse_input() -> List[tuple]:
    """
    Reads the warehouse input from testingdata.txt and stores it into task/item combinations.
    :return: List of task/item combinations.
    """
    f = open('testingdata.txt', 'r')

    result = []
    line = f.readline()
    while line:
        result.append(tuple(line.split()))
        line = f.readline()

    return result


w_input = data_to_warehouse_input()
print(w_input)

[('store', 'red'), ('store', 'blue'), ('store', 'white'), ('restore', 'blue'), ('restore', 'white'), ('store', 'white'), ('store', 'red'), ('store', 'white'), ('store', 'red'), ('restore', 'red'), ('restore', 'red'), ('store', 'white'), ('store', 'blue'), ('store', 'white'), ('restore', 'red'), ('restore', 'white'), ('restore', 'white'), ('store', 'red'), ('store', 'blue'), ('restore', 'red'), ('store', 'blue'), ('store', 'white'), ('restore', 'white'), ('restore', 'blue'), ('restore', 'white'), ('restore', 'white'), ('restore', 'blue'), ('restore', 'blue'), ('store', 'red'), ('restore', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'white'), ('restore', 'red'), ('restore', 'red'), ('restore', 'red'), ('store', 'red'), ('restore', 'white'), ('restore', 'red'), ('restore', 'white'), ('store', 'red'), ('store', 'red'), ('store', 'white'), ('store', 'white'), ('store', 'blue'), ('restore', 'red'), ('restore', 'red'), ('rest

In [595]:
for policy in policies:
    print(distance_mdp(w_input, policy))

print(distance_mdp(w_input, result_policy))
print(distance_greedy(w_input))

120
192
